In [1]:
#Load in the essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations,combinations
from collections import Counter

In [3]:
#create a function that finds pairs and list them in a column. Using A and B as a stand in
def find_pairs(x):
    pairs = pd.DataFrame(list(permutations(x.values,2)),columns=["A","B"])
    return pairs

In [5]:
#Load in the dataset and check the head of the data
dataset=pd.read_csv('Sales_December_2019.csv')
dataset.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Tshirts,1,10.99,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,Jeans,1,78.99,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,Shoes,1,11.95,12-12-2019 18:21,"277 Main St, New York City, NY 10001"
3,295668,Sneakers,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,Shoes,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"


In [6]:
#Group the products by Order ID then apply the function 
dataset_combo =dataset.groupby('Order ID')['Product'].apply(find_pairs).reset_index(drop=True)
dataset_combo.head()

,A,B
0,Suits,Shoes
1,Suits,Belts
2,Suits,Socks
3,Shoes,Suits
4,Shoes,Belts


In [7]:
#  Calculate how often each item item_a occurs with the items in item_b
dataset_combo2 =dataset_combo.groupby(['A','B']).size()
dataset_combo2

A        B           
Belts    Belts            8
         Caps             3
         Deo              2
         Flip Flops       2
         Formal Shoes     6
                         ..
Watches  Shoes           54
         Shorts           1
         Socks           23
         Thermals         1
         Track Pants      1
Length: 234, dtype: int64

In [8]:
#create a sorted dataframe by the most frequent combinations.
dataset =dataset_combo2.reset_index()
dataset.columns = ['A','B',"Size"]
dataset.sort_values(by='Size',ascending =False, inplace =True)
dataset.head()

,A,B,Size
84,Product,Product,2256
152,Suits,Shoes,122
95,Shoes,Suits,122
181,Thermals,Track Pants,116
192,Track Pants,Thermals,116


In [9]:
#Clean the Product row and check the head of the data
Cleaned_Dataset = dataset[dataset["A"]!="Product"]
Cleaned_Dataset.head()

,A,B,Size
152,Suits,Shoes,122
95,Shoes,Suits,122
181,Thermals,Track Pants,116
192,Track Pants,Thermals,116
191,Track Pants,Socks,67


In [10]:
#Create a combination of groups so that can be used as an alternative to pairing
df2=pd.read_csv('Sales_December_2019.csv')
df2= df2.dropna()
df2 =df2[df2['Order ID'].duplicated(keep=False)]
df2['Group'] = df2.groupby('Order ID')['Product'].transform(lambda x:','.join(x))
df2 =df2[['Order ID', 'Group']].drop_duplicates()
df2.head()

,Order ID,Group
16,295681,"Suits,Shoes,Belts,Socks"
36,295698,"Watches,Shoes"
42,295703,"Wallets,Belts"
66,295726,"Track Pants,Thermals"
76,295735,"Track Pants,Shorts,Socks"


In [11]:
#Count each row combination by the the pairing of the two
count = Counter()
for row in df2['Group']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,2)))

print(count)

Counter({('Product', 'Product'): 1128, ('Suits', 'Shoes'): 118, ('Track Pants', 'Thermals'): 113, ('Track Pants', 'Socks'): 66, ('Suits', 'Socks'): 56, ('Watches', 'Shoes'): 53, ('Track Pants', 'Shorts'): 52, ('Suits', 'Belts'): 27, ('Shoes', 'Socks'): 25, ('Watches', 'Socks'): 22, ('Shorts', 'Socks'): 13, ('Shoes', 'Thermals'): 11, ('Thermals', 'Socks'): 11, ('Thermals', 'Wallets'): 11, ('Thermals', 'Shoes'): 11, ('Wallets', 'Thermals'): 10, ('Watches', 'Belts'): 10, ('Belts', 'Socks'): 9, ('Socks', 'Socks'): 9, ('Wallets', 'Wallets'): 9, ('Shoes', 'Belts'): 8, ('Formal Shoes', 'Formal Shoes'): 8, ('Wallets', 'Shorts'): 8, ('Shorts', 'Formal Shoes'): 8, ('Thermals', 'Formal Shoes'): 7, ('Wallets', 'Track Pants'): 7, ('Socks', 'Shoes'): 7, ('Formal Shoes', 'Thermals'): 7, ('Shoes', 'Formal Shoes'): 7, ('Socks', 'Shorts'): 7, ('Thermals', 'Belts'): 6, ('Belts', 'Formal Shoes'): 6, ('Thermals', 'Thermals'): 6, ('Formal Shoes', 'Socks'): 6, ('Wallets', 'Formal Shoes'): 5, ('Socks', 'Walle

In [12]:
#check out the most common combination. 
for key, value in count.most_common(10):
    print(key,value)


('Product', 'Product') 1128
('Suits', 'Shoes') 118
('Track Pants', 'Thermals') 113
('Track Pants', 'Socks') 66
('Suits', 'Socks') 56
('Watches', 'Shoes') 53
('Track Pants', 'Shorts') 52
('Suits', 'Belts') 27
('Shoes', 'Socks') 25
('Watches', 'Socks') 22
